In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import pandas as pd
from tensorflow.keras.applications import ResNet50V2

tf.__version__

'2.3.0-dev20200522'

In [2]:
PATH = 'data'

os.listdir(PATH)

['jpeg224',
 'sample_submission.csv',
 'subset.csv',
 'test.csv',
 'train.csv',
 'train_split.csv',
 'val_split.csv']

In [3]:
train = pd.read_csv(f'{PATH}/subset.csv')
val = pd.read_csv(f'{PATH}/val_split.csv')

train.shape, val.shape

((2220, 8), (10932, 8))

In [4]:
train.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_0533349,IP_5208504,female,45.0,lower extremity,unknown,benign,0
1,ISIC_8814612,IP_0414408,male,50.0,torso,unknown,benign,0
2,ISIC_6515241,IP_6245507,male,45.0,lower extremity,unknown,benign,0
3,ISIC_5075261,IP_2117218,male,40.0,upper extremity,unknown,benign,0
4,ISIC_2624460,IP_1969685,male,50.0,torso,unknown,benign,0


In [11]:
BATCH_SIZE = 64

def decode(filename, label):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32)
    return img, label

def load_ds(df):
    imgs, labels = df["image_name"].values, df["target"].values
    imgs = [f'{PATH}/jpeg224/train/{img}.jpg' for img in imgs]
    ds = tf.data.Dataset.from_tensor_slices((imgs, labels))
    ds = ds.map(decode)
    ds = ds.shuffle(2048)
    ds = ds.batch(BATCH_SIZE)
    return ds

In [12]:
train_ds = load_ds(train)
val_ds = load_ds(val)

In [13]:
IMAGE_SIZE = (224, 224, 3)

encoder = ResNet50V2(
    include_top=False,
    input_shape=IMAGE_SIZE,
    weights='imagenet'
)
encoder.trainable = False

inputs = keras.Input(shape=IMAGE_SIZE)
x = keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)
x = encoder(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
rescaling_1 (Rescaling)      (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50v2 (Functional)      (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2049      
Total params: 23,566,849
Trainable params: 2,049
Non-trainable params: 23,564,800
_________________________________________________________________


In [14]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.AUC(name="auc")]
)

In [16]:
model.fit(train_ds, epochs=10, validation_data=val_ds, validation_steps=10)

Epoch 1/10
35/35 [==============================] - 22s 629ms/step - loss: 0.1863 - auc: 0.4530 - val_loss: 0.1429 - val_auc: 0.4184
Epoch 2/10
35/35 [==============================] - 12s 342ms/step - loss: 0.1028 - auc: 0.5247 - val_loss: 0.0914 - val_auc: 0.6626
Epoch 3/10
35/35 [==============================] - 12s 344ms/step - loss: 0.0865 - auc: 0.6819 - val_loss: 0.1015 - val_auc: 0.4951
Epoch 4/10
35/35 [==============================] - 12s 343ms/step - loss: 0.0779 - auc: 0.7870 - val_loss: 0.1159 - val_auc: 0.6963
Epoch 5/10
35/35 [==============================] - 12s 342ms/step - loss: 0.0719 - auc: 0.8382 - val_loss: 0.0874 - val_auc: 0.7819
Epoch 6/10
35/35 [==============================] - 12s 341ms/step - loss: 0.0673 - auc: 0.8766 - val_loss: 0.1271 - val_auc: 0.7215
Epoch 7/10
35/35 [==============================] - 12s 343ms/step - loss: 0.0640 - auc: 0.8899 - val_loss: 0.1280 - val_auc: 0.7517
Epoch 8/10
35/35 [==============================] - 12s 336ms/step - 

In [17]:
test = pd.read_csv(f'{PATH}/test.csv')

test.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge
0,ISIC_0052060,IP_3579794,male,70.0,NaN
1,ISIC_0052349,IP_7782715,male,40.0,lower extremity
2,ISIC_0058510,IP_7960270,female,55.0,torso
3,ISIC_0073313,IP_6375035,female,50.0,torso
4,ISIC_0073502,IP_0589375,female,45.0,lower extremity


In [18]:
def decode_test(filename):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32)
    return img

def load_test_ds(df):
    imgs = df["image_name"].values
    imgs = [f'{PATH}/jpeg224/test/{img}.jpg' for img in imgs]
    ds = tf.data.Dataset.from_tensor_slices(imgs)
    ds = ds.map(decode_test)
    ds = ds.batch(BATCH_SIZE)
    return ds

In [19]:
test_ds = load_test_ds(test)

In [20]:
preds = model.predict(test_ds)

In [23]:
preds

array([[0.00339877],
       [0.00714143],
       [0.00652743],
       ...,
       [0.03891905],
       [0.00341899],
       [0.15500943]], dtype=float32)

In [24]:
submission = pd.DataFrame({'image_name': test['image_name'].values, 'target': preds.ravel()})

submission

,image_name,target
0,ISIC_0052060,0.003399
1,ISIC_0052349,0.007141
2,ISIC_0058510,0.006527
3,ISIC_0073313,0.001358
4,ISIC_0073502,0.016967
...,...,...
10977,ISIC_9992485,0.010494
10978,ISIC_9996992,0.074677
10979,ISIC_9997917,0.038919
10980,ISIC_9998234,0.003419


In [25]:
submission.to_csv('submission.csv', index=False)